In [1]:
import tensorflow as tf
import mtcnn
from mtcnn import MTCNN
import PIL
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as pyplot

Using TensorFlow backend.


In [2]:
def extract_face(filename):
    #Open the File
    image = Image.open(filename)
    #print(filename)
    #Convert the file to RGB
    image = image.convert('RGB')
    
    #Convert the File to Numpy array to be machine readable
    pixels = np.asarray(image)
    
    #Creating a Face Detector using MTCNN
    detector = MTCNN()
    
    #Extracting Face embeddings from the Photo
    result = detector.detect_faces(pixels)
    #print(result)
    x1, y1, width, height = result[0]['box']
    x1, y1 = abs(x1) , abs(y1)
    x2, y2 = abs(x1) + width , abs(y1) + height 
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((160,160))
    face_array = np.asarray(image)
    return face_array

In [3]:
def load_faces(directory):
    faces = list()
    for filename in os.listdir(directory):
        path = directory + '/' + filename 
        face = extract_face(path)
        faces.append(face)
    return faces

In [4]:
def load_dataset(directory):
    X, y = list(), list()
    for sub_dir in os.listdir(directory):
        path = directory + sub_dir
        faces = load_faces(path)
        print ('Loaded {f} examples from class {c}'.format(f = len(faces), c = sub_dir))
        X.append(faces) 
        y.append(sub_dir)
    return np.asarray(X), np.asarray(y)

In [5]:
X_train, y_train = load_dataset('../data/train/')

Loaded 26 examples from class aditya_solanki
Loaded 14 examples from class ben_afflek
Loaded 17 examples from class elton_john
Loaded 21 examples from class jerry_seinfeld
Loaded 19 examples from class madonna
Loaded 22 examples from class mindy_kaling


In [6]:
X_test, y_test = load_dataset('../data/val/')

Loaded 5 examples from class aditya_solanki
Loaded 5 examples from class ben_afflek
Loaded 5 examples from class elton_john
Loaded 5 examples from class jerry_seinfeld
Loaded 5 examples from class madonna
Loaded 5 examples from class mindy_kaling


In [8]:
np.savez_compressed('test_data.npz',X_train, y_train,X_test, y_test)